#### This notebooks is to generate a database for each indiviudal book, for the purpose of analysis on the four book  on the individual level.

#### import packages

In [25]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import re
import os 
import sqlite3
from scipy.spatial.distance import pdist
%matplotlib inline


import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('tagsets')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/nh4mq/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nh4mq/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/nh4mq/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package tagsets to /home/nh4mq/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package wordnet to /home/nh4mq/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
### Creat eText to Token Function

In [4]:
extra_stopwords = """
us rest went least would much must long one like much say well without though yet might still upon
done every rather particular made many previous always never thy thou go first oh thee ere ye came
almost could may sometimes seem called among another also however nevertheless even way one two three
ever put whose therefore
""".strip().split()

In [5]:
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']
CHAPS = OHCO[:1]
PARAS = OHCO[:2]
SENTS = OHCO[:3]

In [32]:
def get_top_terms(vocab, no_stops=True, sort_col='n', k=1000):
    if no_stops:
        V = vocab[vocab.stop == 0]
    else:
        V = vocab
    return V.sort_values(sort_col, ascending=False).head(k)



def text_to_tokens(name,
                   src_file,
                   body_start=0, 
                   body_end=-1, 
                   chap_pat=r'(^\n*(?:CHAPTER \d+).*$)|(^\n*(?:Chapter \d+).*$)|(^\n*(?:Chapter ((?=[MDCLXVI])M*D?C{0,4}L?X{0,4}V?I{0,4})).*$)|(^\n*(?:CHAPTER ((?=[MDCLXVI])M*D?C{0,4}L?X{0,4}V?I{0,4})).*$)|(^\n*(?:Footnote).*$)|\
            (^\n*(?:INTRODUCTION).*$)|(^\n*(?:PART ((?=[MDCLXVI])M*D?C{0,4}L?X{0,4}V?I{0,4})).*$)|(^\s*(?:CHAPTER \d+).*$)|(^\s*(?:Chapter \d+).*$)|(^\s*(?:Chapter ((?=[MDCLXVI])M*D?C{0,4}L?X{0,4}V?I{0,4})).*$)|\
            (^\n*(?:CHAPTER ((?=[MDCLXVI])M*D?C{0,4}L?X{0,4}V?I{0,4})).*$)|(^\n*(?:Footnote).*$)|\
            (^\s*(?:INTRODUCTION).*$)|(^\s*(?:PART ((?=[MDCLXVI])M*D?C{0,4}L?X{0,4}V?I{0,4})).*$)', 
                   para_pat=r'\n\n+', 
                   sent_pat=r'([.;?!"“”]+)', 
                   token_pat=r'([\W_]+)'):
    db_file=name+'.db'
    # Text to lines
    try:
        lines = open(src_file, 'r', encoding='utf-8').readlines()
    except:
        lines = open(src_file, 'r', encoding='latin-1').readlines()
    df = pd.DataFrame({'line_str':lines})
    df.index.name = 'line_id'
    while df["line_str"][body_start]=="\n":
        body_start+=1
    df=df[body_start:]
    df.reset_index(drop=True,inplace=True)
    del(lines)
    
    # FIX CHARACTERS TO IMPROVE TOKENIZATION
    df.line_str = df.line_str.str.replace('—', ' — ')
    df.line_str = df.line_str.str.replace('-', ' - ')

    # Lines to Chapters
    mask = df.line_str.str.match(chap_pat)
    df.loc[mask, 'chap_id'] = df.apply(lambda x: x.name, 1)
    for i in range(df.shape[0]):
        if bool(re.match('\s*[\r\n]+\s*',df.loc[i]["line_str"])):
            df["chap_id"][i]=np.nan
    df["chap_id"][0]=0
    df.chap_id = df.chap_id.ffill().astype('int')
    chap_ids = df.chap_id.unique().tolist()
    df['chap_num'] = df.chap_id.apply(lambda x: chap_ids.index(x))
    chaps = df.groupby('chap_num')\
        .apply(lambda x: ''.join(x.line_str))\
        .to_frame()\
        .rename(columns={0:'chap_str'})
    del(df)

    # Chapters to Paragraphs
    paras = chaps.chap_str.str.split(para_pat, expand=True)\
        .stack()\
        .to_frame()\
        .rename(columns={0:'para_str'})
    paras.index.names = PARAS
    paras.para_str = paras.para_str.str.strip()
    paras.para_str = paras.para_str.str.replace(r'\n', ' ')
    paras.para_str = paras.para_str.str.replace(r'\s+', ' ')
    paras = paras[~paras.para_str.str.match(r'^\s*$')]
    del(chaps)
    
    # Paragraphs to Sentences
#     sents = paras.para_str.str.split(sent_pat, expand=True)\
    sents = paras.para_str\
        .apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'sent_str'})
    sents.index.names = SENTS
    del(paras)

    # Sentences to Tokens
#     tokens = sents.sent_str.str.split(token_pat, expand=True)\
    tokens = sents.sent_str\
        .apply(lambda x: pd.Series(nltk.pos_tag(nltk.word_tokenize(x))))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'pos_tuple'})
    tokens.index.names = OHCO
    del(sents)

    tokens['pos'] = tokens.pos_tuple.apply(lambda x: x[1])
    tokens['token_str'] = tokens.pos_tuple.apply(lambda x: x[0])
    tokens = tokens.drop('pos_tuple', 1)

    # Tag punctuation and numbers
    tokens['punc'] = tokens.token_str.str.match(r'^[\W_]*$').astype('int')
    tokens['num'] = tokens.token_str.str.match(r'^.*\d.*$').astype('int')
    
    # Extract vocab with minimal normalization
    WORDS = (tokens.punc == 0) & (tokens.num == 0)
    tokens.loc[WORDS, 'term_str'] = tokens.token_str.str.lower()\
        .str.replace(r'["_*.]', '')
    
    vocab = tokens[tokens.punc == 0].term_str.value_counts().to_frame()\
        .reset_index()\
        .rename(columns={'index':'term_str', 'term_str':'n'})
    vocab = vocab.sort_values('term_str').reset_index(drop=True)
    vocab.index.name = 'term_id'
    
    # Get priors for V
    vocab['p'] = vocab.n / vocab.n.sum()
    
    # Add stems
    stemmer = nltk.stem.porter.PorterStemmer()
    vocab['port_stem'] = vocab.term_str.apply(lambda x: stemmer.stem(x))
    
    # Define stopwords
    stopwords = set(nltk.corpus.stopwords.words('english') + extra_stopwords)
    sw = pd.DataFrame({'x':1}, index=stopwords)
    vocab['stop'] = vocab.term_str.map(sw.x).fillna(0).astype('int')
    del(sw)
            
    # Add term_ids to tokens 
    tokens['term_id'] = tokens['term_str'].map(vocab.reset_index()\
        .set_index('term_str').term_id).fillna(-1).astype('int')
    
    with sqlite3.connect(db_file) as db:
        tokens.to_sql('token', db, if_exists='replace', index=True)
        vocab.to_sql('vocab', db, if_exists='replace', index=True)
        
def generate_vsm_related(dbname,alpha=.000001):
    db_name=dbname
    with sqlite3.connect(db_name) as db:
        K = pd.read_sql('SELECT * FROM token', db, index_col=OHCO)
        V = pd.read_sql('SELECT * FROM vocab', db, index_col='term_id')
    WORDS = (K.punc == 0) & (K.num == 0) & K.term_id.isin(V[V.stop==0].index)
    BOW = K[WORDS].groupby(OHCO[:1]+['term_id'])['term_id'].count()
    DTM = BOW.unstack().fillna(0)
    alpha = alpha # We introduce an arbitrary smoothing value
    alpha_sum = alpha * V.shape[0]
    TF = DTM.apply(lambda x: (x + alpha) / (x.sum() + alpha_sum), axis=1)
    N_docs = DTM.shape[0]
    V['df'] = DTM[DTM > 0].count()
    TFIDF = TF * np.log2(N_docs / V[V.stop==0]['df'])
    proper_nouns = K.loc[K.pos == 'NNP', 'term_id'].unique()
    top_n = 1000
    # TOPV = get_top_terms(vocab, sort_col='n')
    TOPV = get_top_terms(V.loc[~V.index.isin(proper_nouns)], sort_col='n')
    tfidf_small = TFIDF[TOPV.index].stack().to_frame().rename(columns={0:'w'})
    THM = -(TF * np.log2(TF))
    TFTH = TF.apply(lambda x: x * THM.sum(), 1)
    V['tf_sum'] = TF.sum()
    V['tf_mean'] = TF.mean()
    V['tf_max'] = TF.max()
    V['tfidf_sum'] = TFIDF.sum()
    V['tfidf_mean'] = TFIDF.mean()
    V['tfidf_max'] = TFIDF.max()
    V['tfth_sum'] = TFTH.sum()
    V['tfth_mean'] = TFTH.mean()
    V['tfth_max'] = TFTH.max()
    V['th_sum'] = THM.sum()
    V['th_mean'] = THM.mean()
    V['th_max'] = THM.max()
    D = DTM.sum(1).astype('int').to_frame().rename(columns={0:'term_count'})
    D['tf'] = D.term_count / D.term_count.sum()

    with sqlite3.connect(db_name) as db:
        V.to_sql('vocab', db, if_exists='replace', index=True)
        K.to_sql('token', db, if_exists='replace', index=True)
        D.to_sql('doc', db, if_exists='replace', index=True)
        BOW.to_frame().rename(columns={'term_id':'n'}).to_sql('bow', db, if_exists='replace', index=True)
        TFIDF.stack().to_frame().rename(columns={0:'term_weight'})\
        .to_sql('dtm_tfidf', db, if_exists='replace', index=True)
        tfidf_small.to_sql('tfidf_small',db,if_exists='replace',index=True)
        
    
def get_docs(tokens, div_names, doc_str = 'term_id', sep='', flatten=False, 
             index_only=False):
    
    if not index_only:
        docs = tokens.groupby(div_names)[doc_str]\
          .apply(lambda x: x.str.cat(sep=sep))
        docs.columns = ['doc_content']
    else:
        docs = tokens.groupby(div_names)[doc_str].apply(lambda x: x.tolist())

    if flatten:
        docs = docs.reset_index().drop(div_names, 1)
    
    return docs

def get_term_id(vocab, term_str):
    return vocab[vocab.term_str == term_str].index[0]

def get_term_str(vocab, term_id):
    return vocab.loc[term_id].term_str    


#### Read in Book Index Dataset

In [12]:
book_index=pd.read_csv("cenlab-index.csv")

In [13]:
drop=book_index.columns[0]
book_index.drop(drop,axis=1,inplace=True)

In [15]:
book_index.head(1)

,author,date,filename,gender,nationality,title,words,modernist
0,Frances Burnett,1881,1881 a fair barbarian.txt,f,a,A Fair Barbarian,41718.0,n


#### Find the file names of the four books

In [16]:
titles= ["The Log of a Cowboy","The Last of the Mohicans","The Outlet","The Mystery of the Cloomber"]

In [20]:
files=book_index[book_index["title"].isin(titles)]["filename"].tolist()

#### Use the src_file and generate database for each book 

In [33]:
for i in files:
    name=i.split('.')[0]
    src_file=r'./texts/'+i
    text_to_tokens(name,src_file)
    
###alll databases have been manually move to a file called database 

/apps/software/standard/core/anaconda/5.2.0-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/apps/software/standard/core/anaconda/5.2.0-py3.6/lib/python3.6/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Genreate BTW, TFIDF, etc.

In [22]:
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']
all_databases = os.listdir('./databases/')   # imagine you're one directory above test dir

In [23]:
for i in all_databases:
    src_file=r'./databases/'+i
    generate_vsm_related(dbname=src_file)

In [26]:
def get_chap_pairs(db_name):
    with sqlite3.connect(db_name) as db:
        D = pd.read_sql('SELECT * FROM doc', db, index_col=['chap_num'])
        TFIDF = pd.read_sql('SELECT * FROM dtm_tfidf',db, index_col=["chap_num","term_id"])
    TFIDF=TFIDF.unstack()
    TFIDF.columns=TFIDF.columns.droplevel(0)
    SIMS = pdist(TFIDF, metric='cosine')
    pair_idx = [(i, j) for i in TFIDF.index for j in TFIDF.index if j > i]
    doc_pairs=pd.DataFrame(columns=["cosine"],index=pd.MultiIndex.from_tuples(pair_idx))
    doc_pairs["cosine"]=SIMS    
    with sqlite3.connect(db_name) as db:
        doc_pairs.to_sql('docpair', db, if_exists='replace', index=True)
   


In [27]:
for i in all_databases:
    src_file=r'./databases/'+i
    get_chap_pairs(db_name=src_file)